Esse notebook faz o tratamento, concatena e mergeia as bases para análise de bolsas do CNPq.

A primeira base 'cnpq_consolidada.csv' é uma query filtrando bolsas da UNICAMP da tabela disponível em: https://console.cloud.google.com/bigquery?p=basedosdados&d=br_cnpq_bolsas&t=microdados&page=table


A segunda base principal, criada a partir da concatenação de diversas 'Painel\_Fomento(tab)\_{year}.csv', foi obtida baixando CSV filtrando as bolsas da UNICAMP por ano do Painel disponível em: http://bi.cnpq.br/painel/fomento-cti/index.html

Autora: Vitória Maria Carneiro Mathias (github: VitoriaMathias)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from google.colab import drive

In [ ]:
pd.set_option('display.max_columns', 500)

# Base consolidada Portal da Transparência CNPq

In [ ]:
# Load data
drive.mount('/content/drive/')
path = '/content/drive/My Drive/MO810 - Data Fem/dados/cnpq_consolidada.csv'

with open(path, "rt") as f:
    df_instituto= pd.read_csv(path)

df_instituto.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,ano,processo,data_inicio_processo,data_fim_processo,beneficiario,titulo_projeto,palavra_chave,linha_fomento,modalidade,categoria_nivel,...,sigla_instituicao_destino,sigla_instituicao_macro,instituicao_destino,plano_interno,unidade_orcamentaria,fonte_recurso,natureza_despesa,programa_ppa,acao_ppa,valor
0,2022,402371/2021-6,2021-12-25,2022-12-31,MARCO ANTONIO CARON RUFFINO,MANUSCRITO,"EPISTEMOLOGIA, FILOSOFIA POLÍTICA, FILOSOFI...",2,AED,NaN,...,UNICAMP,UNICAMP,UNIVERSIDADE ESTADUAL DE CAMPINAS,NaN,CONSELHO NAC.DE DESENV.CIENTIF.E TECNOLOGICO,NaN,339020.0,NaN,NaN,10500.0
1,2022,406514/2021-6,2022-02-04,2025-02-28,LAÍS PELLIZZER GABRIEL,MANUFATURA ADITIVA DE IMPLANTES ODONTOLÓGICOS ...,FIBRAS; MEMBRANA; ÓXIDO DE GRAFENO,4,APQ,NaN,...,UNICAMP,UNICAMP,UNIVERSIDADE ESTADUAL DE CAMPINAS,NaN,FUNDO NACIONAL DE DESENV.CIENT.E TECNOLOGICO,NaN,449020.0,NaN,NaN,31250.0
2,2022,408538/2021-0,2022-03-16,2025-03-31,ARMANDO BOITO JUNIOR,"A CRISE POLÍTICA, A NOVA DIREITA, O ESTADO E O...",NOVA DIREITA; ESTADO; CLASSES SOCIAIS; BRASIL,4,APQ,NaN,...,UNICAMP,UNICAMP,UNIVERSIDADE ESTADUAL DE CAMPINAS,NaN,FUNDO NACIONAL DE DESENV.CIENT.E TECNOLOGICO,NaN,449020.0,NaN,NaN,16000.0
3,2022,409630/2022-5,2022-11-25,2023-11-30,LUIZ FERNANDO ZAGONEL,CONTROLE ELÉTRICO DE EMISSORES DE FÓTONS ÚNICO...,"EMISSÃO SOB DEMANDA. , HBN, EMISSORES DE FÓT...",4,APQ,NaN,...,UNICAMP,UNICAMP,UNIVERSIDADE ESTADUAL DE CAMPINAS,NaN,FUNDO NACIONAL DE DESENV.CIENT.E TECNOLOGICO,NaN,339020.0,NaN,NaN,20000.0
4,2022,403307/2021-0,2022-02-11,2025-02-28,CLARISSA LIN YASUDA,AVALIAÇÃO LONGITUDINAL MULTIMODAL DOS EFEITOS ...,"CAPACIDADE DE TRABALHO, REABILITAÇÃO COGNITI...",4,APQ,NaN,...,UNICAMP,UNICAMP,UNIVERSIDADE ESTADUAL DE CAMPINAS,NaN,FUNDO NACIONAL DE DESENV.CIENT.E TECNOLOGICO,NaN,339020.0,NaN,NaN,82500.0


## Agrupamento

In [ ]:
df_valor_area = df_instituto.groupby(["ano", "modalidade", "area_conhecimento","municipio_destino"], as_index=False).agg({"valor":'sum', "beneficiario":'count'})
df_valor_area.head()

,ano,modalidade,area_conhecimento,municipio_destino,valor,beneficiario
0,2002,AT,MEDICINA,CAMPINAS,5796.12,1
1,2002,AT,MORFOLOGIA,CAMPINAS,4347.09,1
2,2002,GD,AGRONOMIA,CAMPINAS,25749.36,4
3,2002,GD,ANTROPOLOGIA,CAMPINAS,35405.37,4
4,2002,GD,BIOQUÍMICA,CAMPINAS,37551.15,6


Queríamos incluir "Unidade Orçamnetária", mas ela existe apenas para os últimos anos 2021 e 2022

In [ ]:
df_valor_area[["ano", "modalidade", "area_conhecimento", "municipio_destino", "valor"]].to_excel('/content/drive/My Drive/MO810 - Data Fem/dados/valor_cnpq_por_area.xlsx')

# Bases do Painel do CNPq

## Concatenação

In [ ]:
# Load data
# drive.mount('/content/drive/')

df_raca_gen = pd.DataFrame()

for year in np.arange(2005, 2023):
  path = f'/content/drive/My Drive/MO810 - Data Fem/dados/Painel_Fomento(tab)_{year}.csv'

  with open(path, "rt") as f:
      df = pd.read_csv(path, encoding="utf-16", sep='\t',header=1)
  df["ano"] = year

  df_raca_gen = pd.concat([df_raca_gen, df])

df_raca_gen.reset_index(inplace=True, drop=True)
df_raca_gen.head()

,#,01_Instituição,02_Programa,03_Modalidade,04_Cod Categoria Nível,05 _Área,06_Grande Área,07_Linha de Fomento,08_Sexo,09_Cor ou Raça,...,15_Cidade,16_Cod Município IBGE,17_Nacionalidade,18_Naturalidade,Valor (R$),Valor (US$),Benef. Modal / Ano,Bolsa / Ano,Auxílio / Ano,ano
0,1.0,Universidade Estadual de Campinas,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Apoio a Especialista Visitante,D,Engenharia Elétrica,Engenharias,APOIO A PROJETOS DE PESQUISA,Masculino,Branca,...,Campinas,3509502,Brasileira,Holanda,"R$ 11.798,69","$ 5.139,47","1,00","0,00","0,17",2005
1,2.0,Universidade Estadual de Campinas,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Auxílio a Pesquisa,NaN,Ciência da Computação,Ciências Exatas e da Terra,APOIO A PROJETOS DE PESQUISA,Masculino,Não desejo declarar,...,Campinas,3509502,Brasileira,Brasil,"R$ 67.400,00","$ 30.318,02","1,00","0,00","2,00",2005
2,3.0,Universidade Estadual de Campinas,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Auxílio a Pesquisa,NaN,Engenharia Elétrica,Engenharias,APOIO A PROJETOS DE PESQUISA,Masculino,Branca,...,Campinas,3509502,Brasileira,Holanda,"R$ 250.000,00","$ 112.455,58","1,00","0,00","2,00",2005
3,4.0,Universidade Estadual de Campinas,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Desenvolvimento Tecnológico e Industrial,7A,Engenharia Elétrica,Engenharias,APOIO A PROJETOS DE PESQUISA,Masculino,Branca,...,Campinas,3509502,Brasileira,Brasil,"R$ 28.524,33","$ 12.182,78","1,00","0,75","0,00",2005
4,5.0,Universidade Estadual de Campinas,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Desenvolvimento Tecnológico e Industrial,7A,Engenharia Elétrica,Engenharias,APOIO A PROJETOS DE PESQUISA,Masculino,Branca,...,Campinas,3509502,Estrangeira,Alemanha,"R$ 28.524,33","$ 12.182,78","1,00","0,75","0,00",2005


In [ ]:
df_raca_gen['Valor (R$)'][0]

'R$ 11.798,69'

## Tratamento de dados

In [ ]:
def valor_str_to_float(valor_rs: str):
  # Remove 'R$ ' prefix, then remove thousands separator, then replace decimal comma.
  # Handles cases like 'R$ 11.798,69' -> '11798,69' -> '11798.69' -> 11798.69
  return float(valor_rs.replace('R$ ', '').replace('.', '').replace(',', '.'))

df_raca_gen["valor"] = df_raca_gen["Valor (R$)"].apply(lambda x: valor_str_to_float(x))

df_raca_gen[["Valor (R$)", "valor"]].head()

,Valor (R$),valor
0,"R$ 11.798,69",11798.69
1,"R$ 67.400,00",67400.00
2,"R$ 250.000,00",250000.00
3,"R$ 28.524,33",28524.33
4,"R$ 28.524,33",28524.33


## Agrupamento

In [ ]:
df_valor_raca = (
    df_raca_gen.groupby(["ano", "02_Programa", "03_Modalidade", "05 _Área", "08_Sexo", "09_Cor ou Raça", "15_Cidade"], as_index=False)
    .sum("valor")
    )

In [ ]:
df_valor_raca.ano.unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [ ]:
df_valor_raca.head()

,ano,02_Programa,03_Modalidade,05 _Área,08_Sexo,09_Cor ou Raça,15_Cidade,#,16_Cod Município IBGE,valor
0,2005,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Apoio a Especialista Visitante,Engenharia Elétrica,Masculino,Branca,Campinas,1.0,3509502,11798.69
1,2005,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Auxílio a Pesquisa,Ciência da Computação,Masculino,Não desejo declarar,Campinas,2.0,3509502,67400.00
2,2005,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Auxílio a Pesquisa,Engenharia Elétrica,Masculino,Branca,Campinas,3.0,3509502,250000.00
3,2005,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Desenvolvimento Tecnológico e Industrial,Ciência da Computação,Masculino,Não desejo declarar,Campinas,21.0,7019004,8879.58
4,2005,"APOIO A PESQUISA, DESENVOLVIMENTO E INOVACAO E...",Desenvolvimento Tecnológico e Industrial,Engenharia Elétrica,Feminino,Branca,Campinas,14.0,7019004,19396.58


In [ ]:
df_valor_raca[["ano", "02_Programa", "03_Modalidade", "05 _Área", "15_Cidade", "08_Sexo","09_Cor ou Raça", "valor"]].to_excel('/content/drive/My Drive/MO810 - Data Fem/dados/valor_cnpq_por_raca_genero.xlsx')

# Merge das duas tabelas
Formato exclusivamente com as informações que pensamos inicialmente colocar no gráfico

In [ ]:
df_valor1 = df_instituto.groupby(["ano", "area_conhecimento","municipio_destino"], as_index=False).agg({"valor":'sum', "beneficiario":'count'})
df_valor2 = (
    df_raca_gen.groupby(["ano", "05 _Área", "08_Sexo", "09_Cor ou Raça", "15_Cidade"], as_index=False)
    .sum("valor")
    )

In [ ]:
df_valor2["area2"] = df_valor2["05 _Área"].str.upper()
df_valor2["municipio2"] = df_valor2["15_Cidade"].str.upper()
df_valor2.rename(columns = {"valor":"valor2"}, inplace=True)
df_valor2.head()

,ano,05 _Área,08_Sexo,09_Cor ou Raça,15_Cidade,#,16_Cod Município IBGE,valor2,area2,municipio2
0,2005,Administração,Feminino,Amarela,Campinas,1.329,3509502,900.00,ADMINISTRAÇÃO,CAMPINAS
1,2005,Administração,Feminino,Branca,Campinas,1689.062,17547510,25704.24,ADMINISTRAÇÃO,CAMPINAS
2,2005,Administração,Feminino,Não desejo declarar,Campinas,2.558,7019004,7160.58,ADMINISTRAÇÃO,CAMPINAS
3,2005,Administração,Feminino,Parda,Campinas,1.229,3509502,8747.44,ADMINISTRAÇÃO,CAMPINAS
4,2005,Administração,Masculino,Branca,Campinas,1205.966,28076016,109906.70,ADMINISTRAÇÃO,CAMPINAS


In [ ]:
df_consolidado = df_valor1.merge(df_valor2, how='left', left_on=["ano","area_conhecimento","municipio_destino"], right_on = ["ano","area2","municipio2"])
df_consolidado.head()

,ano,area_conhecimento,municipio_destino,valor,beneficiario,05 _Área,08_Sexo,09_Cor ou Raça,15_Cidade,#,16_Cod Município IBGE,valor2,area2,municipio2
0,2002,ADMINISTRAÇÃO,CAMPINAS,24340.80,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,AGRONOMIA,CAMPINAS,51685.00,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,ANTROPOLOGIA,CAMPINAS,116803.85,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,ARTES,CAMPINAS,18837.56,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,BIOQUÍMICA,CAMPINAS,124589.63,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_consolidado.to_excel('/content/drive/My Drive/MO810 - Data Fem/dados/valor_cnpq_consolidado.xlsx')

## Adicionando Institutos com De-Para feito à mão a partir da Área de Conhecimento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
path_dados = '/content/drive/My Drive/MO810 - Data Fem/dados'
path_instituto = '/de_para_instituto.xlsx'
path_consolidado = '/valor_cnpq_consolidado.xlsx'
df_de_para = pd.read_excel(path_dados+path_instituto)
df_consolidado = pd.read_excel(path_dados+path_consolidado)

In [ ]:
df_consol_atual = df_consolidado.merge(df_de_para, left_on=['area_conhecimento', 'municipio_destino'], right_on=["area","cidade"], how='left')
display(df_consol_atual.head())

,Unnamed: 0,ano,area_conhecimento,municipio_destino,valor,beneficiario,05 _Área,08_Sexo,09_Cor ou Raça,15_Cidade,#,16_Cod Município IBGE,valor2,area2,municipio2,area,instituto,cidade
0,0,2002,ADMINISTRAÇÃO,CAMPINAS,24340.80,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADMINISTRAÇÃO,IE,CAMPINAS
1,1,2002,AGRONOMIA,CAMPINAS,51685.00,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRONOMIA,FEAGRI,CAMPINAS
2,2,2002,ANTROPOLOGIA,CAMPINAS,116803.85,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANTROPOLOGIA,IFHC,CAMPINAS
3,3,2002,ARTES,CAMPINAS,18837.56,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARTES,IA,CAMPINAS
4,4,2002,BIOQUÍMICA,CAMPINAS,124589.63,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BIOQUÍMICA,IB,CAMPINAS


In [ ]:
df_consol_atual.to_excel('/content/drive/My Drive/MO810 - Data Fem/dados/valor_cnpq_consolidado_v2.xlsx')